# notebook to analyze axon data after suite2p

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from matplotlib import mlab
from scipy.signal import detrend

In [ ]:
F=np.load('F.npy')
iscell=np.load('iscell.npy',allow_pickle=True)[:,0]
stat=np.load('stat.npy',allow_pickle=True)

F.shape, iscell.shape, stat.shape

In [3]:
#get the cell ids
ncells=len(stat)
cell_ids = np.arange(ncells)

#detect trace data of cells
condition = np.mod(iscell, 2)==1
cells=np.extract(condition,cell_ids)
F_cell=F[cells,:]

## Calculate each dff and mean dff from each dff

In [ ]:
rois = np.arange(len(F_cell))[::1]  
dff_f_each = np.empty(shape=F_cell.shape)
for roi in rois:
    meanF=F_cell[roi]
    meanF0=np.percentile(meanF, 8)
    mean_dff = 100*(meanF-meanF0)/meanF0
    mean_dff_f = savgol_filter(mean_dff,50,3)
    index = roi-1
    dff_f_each[index] = mean_dff_f

dff_f_each

In [ ]:
sum_dff=0
for roi in rois:
    sum_dff=sum_dff+dff_f_each[roi]
dff_f_mean_from_each_dff=sum_dff/len(F_cell)
dff_f_mean_from_each_dff

In [18]:
np.save('dff_f_each.npy',dff_f_each)
np.save('dff_f_mean_from_each_dff.npy',dff_f_mean_from_each_dff)

In [ ]:
dff_f_each=np.load('dff_f_each.npy')
dff_f_mean_from_each_dff=np.load('dff_f_mean_from_each_dff.npy')
dff_f_each.shape,dff_f_mean_from_each_dff.shape

### detect the time of teleport and switch env

In [ ]:
#Load downsampled behavioral data
df_downsample=pd.read_csv('df_downsample.csv')

In [ ]:
# Finding the time teleport back
n = np.array(range(0,len(df_downsample)-1,1))
teleport_list=[]
for u in n: 
    if df_downsample.Position[u]-df_downsample.Position[u+1] > 5:
        if df_downsample.Position[u] > 5:
            teleport_list.append(u)
                  
teleport_list

In [86]:
teleport_list_230706_C7=np.array(teleport_list)
np.save('teleport_list_240804_C20.npy',teleport_list_230706_C7)

In [31]:
# Calculating the cell swicthing environment
n = np.array(range(0,len(df_downsample)-1,1))
for u in n: 
    if abs(df_downsample.Environment[u+1]-df_downsample.Environment[u]) > 1:
        print(u+1)

9386
28491


In [4]:
switch1=9386
switch2=28491

In [34]:
teleport_list.index(9380)

21

In [35]:
teleport_list[21]

9380

In [36]:
teleport_list.index(28485)

55

In [38]:
teleport_list[55]

28485

In [39]:
switch_heatmap1=teleport_list.index(9380)-0.5
switch_heatmap2=teleport_list.index(28485)-0.5

In [42]:
# Define the column names
column_names = ["switch1", "switch2", "switch_heatmap1", "switch_heatmap2"]

# Organize switch values
data = {
    "switch1": switch1,  # Replace value1 with your actual value
    "switch2": switch2,  # Replace value2 with your actual value
    "switch_heatmap1": switch_heatmap1,  # Replace value3 with your actual value
    "switch_heatmap2": switch_heatmap2,  # Replace value4 with your actual value
}

# Create the DataFrame
switch_240605_C14H = pd.DataFrame(data,index=[0])

# Display the DataFrame
switch_240605_C14H

,switch1,switch2,switch_heatmap1,switch_heatmap2
0,9386,28491,20.5,54.5


In [43]:
switch_240605_C14H.to_csv('switch_240804_C20.csv',index=False)  

### Inspection of signals by plotting graphs

In [ ]:
#Mean_trace + Velocity
fig, ax1 = plt.subplots(figsize=[13,5])
plt.suptitle("240804_C20_mean_trace_env_swicth",y=1.1, fontsize='xx-large')    
    
ax1.plot(df_downsample.Time, df_downsample.Velocity,   color='#0344DF', linewidth=0.1)

plt.axvline(x=df_downsample.Time[switch1])
plt.axvline(x=df_downsample.Time[switch2])
ax1.axvspan(0,df_downsample.Time[switch1-1], facecolor='lightblue', alpha=0.65)
ax1.axvspan(df_downsample.Time[switch1+1],df_downsample.Time[switch2-1], facecolor='chartreuse', alpha=0.4)
ax1.axvspan(df_downsample.Time[switch2+1],df_downsample.Time[len(dff_f_mean_from_each_dff)-1], facecolor='lightblue', alpha=0.4)

ax2 = ax1.twinx()
ax2.plot(df_downsample.Time, dff_f_mean_from_each_dff, color='tab:orange',label="Mean Trace", linewidth=0.6)

plt.tight_layout()  # otherwise the right y-label is slightly clipped
    
plt.legend(bbox_to_anchor=(0.93, 1.1)) 

### Check speed across environments

In [ ]:
# average speed

br1=df_downsample.Velocity[:switch1-1].mean()
br2=df_downsample.Velocity[switch1+1:switch2-1].mean()
br3=df_downsample.Velocity[switch2+1:].mean()

print(br1)
print(br2)
print(br3)

In [ ]:
#Monitoring running speed across environments

from matplotlib.lines import Line2D
plt.bar(0,br1, color ='blue', alpha=0.4, 
        edgecolor ='grey', label ='env1') 
plt.bar(1,br2,  color ='green', alpha=0.4, 
        edgecolor ='grey', label ='env2') 
plt.bar(2,br3,  color ='midnightblue', alpha=0.4, 
        edgecolor ='grey', label ='returned env1') 

plt.suptitle("240804_C20_mean_velocity",y=1.0, fontsize='xx-large')

#remove x ticks
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    top=False,         # ticks along the top edge are off
    labelbottom=False) 

legend_handles = [Line2D([0], [0], color='blue', alpha=0.75,label='familiar'),
                  Line2D([0], [0], color='green', alpha=0.75,label='novel'),
                  Line2D([0], [0], color='midnightblue', alpha=0.75,label='familiar_returned')]

# Add the legend with custom handles and labels
plt.legend(handles=legend_handles,bbox_to_anchor=(1, 1.1))


plt.text(-0.1, br1, str(round(br1, 3)))
plt.text(0.9, br2, str(round(br2, 3)))
plt.text(1.9, br3, str(round(br3, 3)))


# predict dff with log fit to speed vs dff

In [15]:
velocity_f = savgol_filter(df_downsample.Velocity,50,3) # savgol_filter for velocity
velocity_f=pd.DataFrame(velocity_f)
velocity_f_positive=velocity_f[0]-velocity_f[0].min()+0.0000001 # Adjust velocity by deducting min_vel

In [28]:
#for average trace
pd_dff_f_mean_eachdff=pd.DataFrame(dff_f_mean_from_each_dff)[0]
dff_f_mean_env1=pd_dff_f_mean_eachdff[:switch1-1] 
scale_factor = 100 / (np.percentile(dff_f_mean_env1,90) - min(dff_f_mean_env1))
rescaled_dff = (pd_dff_f_mean_eachdff - min(dff_f_mean_env1)) * scale_factor

x = velocity_f_positive[:switch1]  # Reverse the order
y = rescaled_dff[:switch1]   # Reverse the order   
coeffs = np.polyfit(np.log(x), y, 1)
x = velocity_f_positive
ypred = np.polyval(coeffs, np.log(x))
novelty_average=rescaled_dff-ypred

In [60]:
#Actual dff data - prediction (normalized)
pd_dff_f_mean_eachdff=pd.DataFrame(dff_f_mean_from_each_dff)[0]
dff_f_mean_env1=pd_dff_f_mean_eachdff[:switch1-1] 
scale_factor = 100 / (np.percentile(dff_f_mean_env1,90) - min(dff_f_mean_env1))
rescaled_dff = (pd_dff_f_mean_eachdff - min(dff_f_mean_env1)) * scale_factor

x = velocity_f_positive[:switch1]  # Reverse the order
y = rescaled_dff[:switch1]   # Reverse the order   
coeffs = np.polyfit(np.log(x), y, 1)
x = velocity_f_positive
ypred = np.polyval(coeffs, np.log(x))
novelty_average=rescaled_dff-ypred
np.save('novelty_average_240804_C20',novelty_average)
novelty_average

0       -91.166301
1       -80.656885
2       -70.996820
3       -62.156345
4       -54.105476
           ...    
37979   -24.829087
37980   -24.122907
37981   -23.240882
37982   -22.145291
37983   -20.795990
Name: 0, Length: 37984, dtype: float64

In [ ]:
#min_average of the prediction error in env1
novelty_average_minute_familiar = []

for n in range(1,10+1):
    if n == 1:
        # Compute the absolute difference between the 'Time' column and 1
        abs_diff = np.abs(df_downsample.Time - n*60)
        # Find the index of the minimum value in the absolute difference series
        X = np.argmin(abs_diff)
        novelty_minute=novelty_average[:switch1][:X+1].mean()
        novelty_average_minute_familiar.append(novelty_minute) 
    else:
        abs_diff1 = np.abs(df_downsample.Time - (n-1)*60)
        X1 = np.argmin(abs_diff1)
        abs_diff2 = np.abs(df_downsample.Time - n*60)
        X2 = np.argmin(abs_diff2)
        novelty_minute=novelty_average[:switch1][X1:X2+1].mean()
        novelty_average_minute_familiar.append(novelty_minute) 
        
print(novelty_average_minute_familiar, len(novelty_average_minute_familiar))

In [ ]:
#min_average of the prediction error in env2
novelty_average_minute_novel = []

for n in range(1,20+1):
    if n == 1:
        # Compute the absolute difference between the 'Time' column and 1
        abs_diff = np.abs(df_downsample.Time - n*60)
        # Find the index of the minimum value in the absolute difference series
        X = np.argmin(abs_diff)
        novelty_minute=novelty_average[switch1:switch2][:X+1].mean()
        novelty_average_minute_novel.append(novelty_minute) 
    else:
        abs_diff1 = np.abs(df_downsample.Time - (n-1)*60)
        X1 = np.argmin(abs_diff1)
        abs_diff2 = np.abs(df_downsample.Time - n*60)
        X2 = np.argmin(abs_diff2)
        novelty_minute=novelty_average[switch1:switch2][X1:X2+1].mean()
        novelty_average_minute_novel.append(novelty_minute) 
        
print(novelty_average_minute_novel, len(novelty_average_minute_novel))

In [ ]:
#min_average of the prediction error in returned env1
novelty_average_minute_Rfamiliar = []

for n in range(1,10+1):
    if n == 1:
        # Compute the absolute difference between the 'Time' column and 1
        abs_diff = np.abs(df_downsample.Time - n*60)
        # Find the index of the minimum value in the absolute difference series
        X = np.argmin(abs_diff)
        novelty_minute=novelty_average[switch2:][:X+1].mean()
        novelty_average_minute_Rfamiliar.append(novelty_minute) 
    else:
        abs_diff1 = np.abs(df_downsample.Time - (n-1)*60)
        X1 = np.argmin(abs_diff1)
        abs_diff2 = np.abs(df_downsample.Time - n*60)
        X2 = np.argmin(abs_diff2)
        novelty_minute=novelty_average[switch2:][X1:X2+1].mean()
        novelty_average_minute_Rfamiliar.append(novelty_minute) 
        
print(novelty_average_minute_Rfamiliar, len(novelty_average_minute_Rfamiliar))

In [12]:
novelty_average=np.load('novelty_average_240804_C20.npy')

In [81]:
np.save('novelty_average_minute_familiar_240804_C20',novelty_average_minute_familiar)
np.save('novelty_average_minute_novel_240804_C20',novelty_average_minute_novel)
np.save('novelty_average_minute_Rfamiliar_240804_C20',novelty_average_minute_Rfamiliar)

In [ ]:
# plot speed vs prediction error in env1
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# set the data
x = velocity_f_positive[switch1-2:0:-1]  # Reverse the order
y = rescaled_dff[switch1-2:0:-1]  # Reverse the order 

spectral = plt.cm.get_cmap('Spectral')
half_spectral = ListedColormap(spectral(np.linspace(0, 0.5, 256)))

fig, ax = plt.subplots(figsize=[5.4,5])
scatter=ax.scatter(x, y, c=df_downsample.Time[switch1-2:0:-1], s=1, cmap=half_spectral)
plt.ylabel("normalized dF/F", fontsize=14)
plt.xlabel("Velocity", fontsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
plt.ylim(0,150)
plt.plot(x, ypred[switch1-2:0:-1], color="red")     # regression line

axins = inset_axes(ax,
                   width="3%",  # width of the colorbar
                   height="50%",  # height of the colorbar
                   loc='center right',
                   bbox_to_anchor=(0.1, 0.0, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )
cb = plt.colorbar(scatter, cax=axins)
cb.set_label('Time',rotation=270, fontsize=14)
tick_locs = [df_downsample.Time[0],df_downsample.Time[switch1-1]]  # Adjust these based on your data range
tick_lbls = ['Start','End']
cb.ax.set_yticks(tick_locs)    # Set the tick locations
cb.ax.set_yticklabels(tick_lbls, fontsize=14)  # Set the tick labels
cb.ax.invert_yaxis()# Reverse the direction of the colorbar

plt.show()

In [ ]:
# plot speed vs prediction error in env2
x = velocity_f_positive[switch2-2:switch1:-1]  # Reverse the order
y = rescaled_dff[switch2-2:switch1:-1]   # Reverse the order   

fig, ax = plt.subplots(figsize=[5.4,5])
scatter=ax.scatter(x, y, c=df_downsample.Time[switch2-2:switch1:-1], s=0.6, cmap='Spectral')
plt.ylabel("normalized dF/F", fontsize=14)
plt.xlabel("Velocity", fontsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
plt.ylim(0,150)
plt.plot(x, ypred[switch2-2:switch1:-1], color="red")     # regression line

axins = inset_axes(ax,
                   width="3%",  # width of the colorbar
                   height="100%",  # height of the colorbar
                   loc='center right',
                   bbox_to_anchor=(0.1, 0.0, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )
cb = plt.colorbar(scatter, cax=axins)
cb.set_label('Time',rotation=270, fontsize=14,labelpad=20)
mid_point=(df_downsample.Time[switch2-1]-df_downsample.Time[switch1+1])/2+df_downsample.Time[switch1+1]
tick_locs = [df_downsample.Time[switch1+1],mid_point, df_downsample.Time[switch2-1]]  # Adjust these based on your data range
tick_lbls = ['0 min','10 min','20 min']
cb.ax.set_yticks(tick_locs)    # Set the tick locations
cb.ax.set_yticklabels(tick_lbls, fontsize=14)  # Set the tick labels
cb.ax.invert_yaxis()# Reverse the direction of the colorbar

plt.show()

In [ ]:
# plot speed vs prediction error in returned env1
x = velocity_f_positive[len(velocity_f_positive)-1:switch2:-1]  # Reverse the order
y = rescaled_dff[len(velocity_f_positive)-1:switch2:-1]  # Reverse the order 

spectral = plt.cm.get_cmap('Spectral')
half_spectral = ListedColormap(spectral(np.linspace(0, 0.5, 256)))

fig, ax = plt.subplots(figsize=[5.4,5])
scatter=ax.scatter(x, y, c=df_downsample.Time[len(velocity_f_positive)-1:switch2:-1], s=1, cmap=half_spectral)

plt.ylabel("normalized dF/F", fontsize=14)
plt.xlabel("Velocity", fontsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
plt.ylim(0,150)
plt.plot(x, ypred[len(velocity_f_positive)-1:switch2:-1], color="red")     # regression line

axins = inset_axes(ax,
                   width="3%",  # width of the colorbar
                   height="50%",  # height of the colorbar
                   loc='center right',
                   bbox_to_anchor=(0.1, 0.0, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )
cb = plt.colorbar(scatter, cax=axins)
cb.set_label('Time',rotation=270, fontsize=14)
tick_locs = [df_downsample.Time[switch2+1],df_downsample.Time.iloc[-1]]  # Adjust these based on your data range
tick_lbls = ['Start','End']
cb.ax.set_yticks(tick_locs)    # Set the tick locations
cb.ax.set_yticklabels(tick_lbls, fontsize=14)  # Set the tick labels
cb.ax.invert_yaxis()# Reverse the direction of the colorbar

plt.show()

## Xcorr_average_minute

### Define cross-correlation with manual set lag

In [58]:
from numpy.lib.stride_tricks import as_strided


def _check_arg(x, xname):
    x = np.asarray(x)
    if x.ndim != 1:
        raise ValueError('%s must be one-dimensional.' % xname)
    return x

def autocorrelation(x, maxlag):
    """
    Autocorrelation with a maximum number of lags.

    `x` must be a one-dimensional numpy array.

    This computes the same result as
        numpy.correlate(x, x, mode='full')[len(x)-1:len(x)+maxlag]

    The return value has length maxlag + 1.
    """
    x = _check_arg(x, 'x')
    p = np.pad(x.conj(), maxlag, mode='constant')
    T = as_strided(p[maxlag:], shape=(maxlag+1, len(x) + maxlag),
                   strides=(-p.strides[0], p.strides[0]))
    return T.dot(p[maxlag:].conj())


def crosscorrelation(x, y, maxlag):
    """
    Cross correlation with a maximum number of lags.

    `x` and `y` must be one-dimensional numpy arrays with the same length.

    This computes the same result as
        numpy.correlate(x, y, mode='full')[len(a)-maxlag-1:len(a)+maxlag]

    The return vaue has length 2*maxlag + 1.
    """
    x = _check_arg(x, 'x')
    y = _check_arg(y, 'y')
    py = np.pad(y.conj(), 2*maxlag, mode='constant')
    T = as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
                   strides=(-py.strides[0], py.strides[0]))
    px = np.pad(x, maxlag, mode='constant')
    return T.dot(px)

In [ ]:
#min_scale_correlation to speed in env1
xcorr_average_minute_familiar = []

for n in range(1,10+1):
    if n == 1:
        # Compute the absolute difference between the 'Time' column and 1
        abs_diff = np.abs(df_downsample.Time - n*60)
        # Find the index of the minimum value in the absolute difference series
        X = np.argmin(abs_diff)
        data1 = dff_f_mean_from_each_dff[:switch1][:X+1]
        data2 = velocity_f_positive[:switch1][:X+1]
        # Detrend the data
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 15) #15 frames~ 1sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        # Find the maximum correlation coefficient
        max_corr_coefficient = np.max(normalized_corr)
        xcorr_average_minute_familiar.append(max_corr_coefficient) 
    else:
        abs_diff1 = np.abs(df_downsample.Time - (n-1)*60)
        X1 = np.argmin(abs_diff1)
        abs_diff2 = np.abs(df_downsample.Time - n*60)
        X2 = np.argmin(abs_diff2)
        data1 = dff_f_mean_from_each_dff[:switch1][X1:X2+1]
        data2 = velocity_f_positive[:switch1][X1:X2+1]
        # Detrend the data
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 15) #15 frames~ 1sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        # Find the maximum correlation coefficient
        max_corr_coefficient = np.max(normalized_corr)
        xcorr_average_minute_familiar.append(max_corr_coefficient) 

print(xcorr_average_minute_familiar, len(xcorr_average_minute_familiar))

In [ ]:
#min_scale_correlation to speed in env2
xcorr_average_minute_novel = []

for n in range(1,20+1):
    if n == 1:
        # Compute the absolute difference between the 'Time' column and 1
        abs_diff = np.abs(df_downsample.Time - n*60)
        # Find the index of the minimum value in the absolute difference series
        X = np.argmin(abs_diff)
        data1 = dff_f_mean_from_each_dff[switch1:switch2][:X+1]
        data2 = velocity_f_positive[switch1:switch2][:X+1]
        # Detrend the data
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 15) #15 frames~ 1sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        # Find the maximum correlation coefficient
        max_corr_coefficient = np.max(normalized_corr)
        xcorr_average_minute_novel.append(max_corr_coefficient)
    else:
        abs_diff1 = np.abs(df_downsample.Time - (n-1)*60)
        X1 = np.argmin(abs_diff1)
        abs_diff2 = np.abs(df_downsample.Time - n*60)
        X2 = np.argmin(abs_diff2)
        data1 = dff_f_mean_from_each_dff[switch1:switch2][X1:X2+1]
        data2 = velocity_f_positive[switch1:switch2][X1:X2+1]
        # Detrend the data
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 15) #15 frames~ 1sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        # Find the maximum correlation coefficient
        max_corr_coefficient = np.max(normalized_corr)
        xcorr_average_minute_novel.append(max_corr_coefficient)    

print(xcorr_average_minute_novel, len(xcorr_average_minute_novel))

In [ ]:
#min_scale_correlation to speed in returned env1
xcorr_average_minute_Rfamiliar = []

for n in range(1,10+1):
    if n == 1:
        # Compute the absolute difference between the 'Time' column and 1
        abs_diff = np.abs(df_downsample.Time - n*60)
        # Find the index of the minimum value in the absolute difference series
        X = np.argmin(abs_diff)
        data1 = dff_f_mean_from_each_dff[switch2:][:X+1]
        data2 = velocity_f_positive[switch2:][:X+1]
        # Detrend the data
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 15) #15 frames~ 1sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        # Find the maximum correlation coefficient
        max_corr_coefficient = np.max(normalized_corr)
        xcorr_average_minute_Rfamiliar.append(max_corr_coefficient) 
    else:
        abs_diff1 = np.abs(df_downsample.Time - (n-1)*60)
        X1 = np.argmin(abs_diff1)
        abs_diff2 = np.abs(df_downsample.Time - n*60)
        X2 = np.argmin(abs_diff2)
        data1 = dff_f_mean_from_each_dff[switch2:][X1:X2+1]
        data2 = velocity_f_positive[switch2:][X1:X2+1]
        # Detrend the data
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 15) #15 frames~ 1sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        # Find the maximum correlation coefficient
        max_corr_coefficient = np.max(normalized_corr)
        xcorr_average_minute_Rfamiliar.append(max_corr_coefficient) 

print(xcorr_average_minute_Rfamiliar, len(xcorr_average_minute_Rfamiliar))

In [80]:
np.save('xcorr_minute_familiar_240804_C20',np.array(xcorr_average_minute_familiar))
np.save('xcorr_minute_novel_240804_C20',np.array(xcorr_average_minute_novel))
np.save('xcorr_minute_Rfamiliar_240804_C20',np.array(xcorr_average_minute_Rfamiliar))

In [ ]:
#Overall_Cross correlation_between 3env
from matplotlib import mlab
from matplotlib.lines import Line2D

xcor_value0 = plt.xcorr(pd.DataFrame(dff_f_mean_from_each_dff)[0][:switch1], 
                       velocity_f_positive[:switch1],
                       detrend=mlab.detrend_linear, 
                       maxlags=300,
                       color='blue',
                       alpha=0.1
                       )

xcor_value2 = plt.xcorr(pd.DataFrame(dff_f_mean_from_each_dff)[0][switch1+1:switch2-1], 
                       velocity_f_positive[switch1+1:switch2-1],
                       detrend=mlab.detrend_linear, 
                       maxlags=300,
                       color='green',
                       alpha=0.1)

xcor_value5 = plt.xcorr(pd.DataFrame(dff_f_mean_from_each_dff)[0][switch2+1:], 
                       velocity_f_positive[switch2+1:],
                       detrend=mlab.detrend_linear, 
                       maxlags=300,
                       color='midnightblue',
                       alpha=0.1
                       )


custom_xtick_positions = [-309.8,0,309.8]
custom_xtick_labels = ['-20','0','20']
plt.xticks(custom_xtick_positions,custom_xtick_labels)
plt.xlabel("sec")
plt.suptitle("240804_C20_dff_crosscorrelation",y=1.0, fontsize='xx-large')
plt.ylim(-0.7,1)

# Create custom legend handles and labels
legend_handles = [Line2D([0], [0], color='blue', alpha=0.75,label='env1'),
                  Line2D([0], [0], color='green', alpha=0.75,label='env2'),
                  Line2D([0], [0], color='midnightblue', alpha=0.75,label='returned_env1')]

# Add the legend with custom handles and labels
#plt.legend(handles=legend_handles)

plt.show()

In [ ]:
# Example Trace with velocity&Lick

fig, ax1 = plt.subplots(figsize=[10,4])

ax1.plot(df_downsample.Velocity[2000:5000], 
        
         color='tab:blue', linewidth=0.75)


ax2 = ax1.twinx()
ax2.plot(pd.DataFrame(dff_f_mean_from_each_dff)[2000:5000], 
          color='orange',
        linewidth=2)
ax2.plot(dfl_downsample.Lick[2000:5000], 
        
         color='magenta', linewidth=2)
plt.suptitle("240804_C20_2000-5000_mean_velocity",y=0.95, fontsize='xx-large')
ax1.axvspan(2000,5000, facecolor='lightblue', alpha=0.4)
fig.tight_layout()

In [ ]:
# Example Trace with velocity&Reward

fig, ax1 = plt.subplots(figsize=[10,4])

ax1.plot(df_downsample.Velocity[2000:5000], 
        
         color='tab:blue', linewidth=0.75)


ax2 = ax1.twinx()
ax2.plot(pd.DataFrame(dff_f_mean_from_each_dff)[2000:5000], 
          color='orange',
        linewidth=2)
ax2.plot(dfl_downsample.Lick[2000:5000],         
         color='magenta', linewidth=2)
ax2.plot(dfl_downsample.Reward[2000:5000],         
         color='green', linewidth=2)
plt.suptitle("240804_C20_2000-5000_mean_velocity",y=0.95, fontsize='xx-large')
ax1.axvspan(2000,5000, facecolor='lightblue', alpha=0.4)
fig.tight_layout()